In [12]:
# import necessary libs
import numpy as np
import pandas as pd
from scipy import optimize
import scipy as sp
import sympy as sy
from datetime import datetime
import matplotlib.pyplot as plt

In [8]:
# Set right font display style
plt.rcParams["font.sans-serif"]=["Times New Roman"]
plt.rcParams["axes.unicode_minus"]=False
# Read file.
# df contains all the data.
df = pd.read_excel('./raw_data.xlsx', header = 1)
df = df[df.columns[1:]]
# display the data
df.head()

,Date,Contest number,Word,Number of reported results,Number in hard mode,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X)
0,2022-12-31,560,manly,20380,1899,0,2,17,37,29,12,2
1,2022-12-30,559,molar,21204,1973,0,4,21,38,26,9,1
2,2022-12-29,558,havoc,20001,1919,0,2,16,38,30,12,2
3,2022-12-28,557,impel,20160,1937,0,3,21,40,25,9,1
4,2022-12-27,556,condo,20879,2012,0,2,17,35,29,14,3


In [23]:
# data pre-processing
# delete the data which is extremly different from the siblings
## set the sum of the possibility of tries to one.
df = df.sort_values(by='Contest number', ignore_index=True)
percent = df[df.columns[5:]].sum(axis=1)
for column in df.columns[5:]:
    df[column]=df[column]/percent*100
## counting the word_length of the set.  
df['word_len'] = df['Word'].apply(lambda x:len(set(x)))
## calculating the hard mode rate.
df['hard_mode_rate'] = df['Number in hard mode']/df['Number of  reported results']
df.head()
## delete the data which rates is extremely high
### Z-Score mode.
df_zscore = df.copy()  
cols = df.columns[5:]  
for col in cols: 
    df_col = df[col]  
    z_score = (df_col - df_col.mean()) / df_col.std() 
    df_zscore[col] = z_score.abs() > 2
print(df_zscore)
cols = df.columns[5:]  
for col in cols: 
    df[df_zscore[col] == False]
df.head()

          Date  Contest number   Word  Number of  reported results  \
0   2022-01-07             202  slump                        80630   
1   2022-01-08             203  crank                       101503   
2   2022-01-09             204  gorge                        91477   
3   2022-01-10             205  query                       107134   
4   2022-01-11             206  drink                       153880   
..         ...             ...    ...                          ...   
354 2022-12-27             556  condo                        20879   
355 2022-12-28             557  impel                        20160   
356 2022-12-29             558  havoc                        20001   
357 2022-12-30             559  molar                        21204   
358 2022-12-31             560  manly                        20380   

     Number in hard mode  1 try  2 tries  3 tries  4 tries  5 tries  6 tries  \
0                   1362  False    False    False    False    False    False   

,Date,Contest number,Word,Number of reported results,Number in hard mode,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X),word_len,hard_mode_rate
0,2022-01-07,202,slump,80630,1362,0.949831,2.849493,21.846115,37.043412,22.795946,8.548480,0.949831,5,0.016892
1,2022-01-08,203,crank,101503,1763,0.949826,4.749132,21.846005,29.444616,22.795831,13.297568,1.899653,5,0.017369
2,2022-01-09,204,gorge,91477,1913,0.959791,2.879373,12.477281,25.914354,28.793726,21.115399,3.839164,4,0.020912
3,2022-01-10,205,query,107134,2242,0.949791,3.799163,15.196652,28.493722,28.493722,16.146442,1.899581,5,0.020927
4,2022-01-11,206,drink,153880,3017,0.940400,8.463599,32.913998,31.973598,15.046399,4.702000,0.940400,5,0.019606
